In [51]:
import os
import numpy as np
import pandas as pd
import pupil_recording_interface as pri
from hvplot import xarray
import yaml
import decimal
import datetime
import math
from scipy.signal import hilbert
from scipy.signal import find_peaks,find_peaks_cwt
from sklearn.linear_model import LinearRegression
from scipy.stats import variation
import matplotlib.pyplot as plt
%matplotlib notebook

## Read in odometry data

In [52]:
location = "/media/bszekely/BrianDrive/VEDB_data/slippage_data/V1/"
session_num = "2021_07_14_17_30_31"
recording = location + session_num
world_time_stamp_file = location + session_num + "/world_timestamps.npy"
odometry_time_stamp_file = location + session_num + "/odometry_timestamps.npy"
#valref_time_stamp_file = location + session_num + "/validation_found.npy"
world_time_stamp = np.load(world_time_stamp_file)
odometry_time_stamp = np.load(odometry_time_stamp_file)
#valref_time_stamp = np.load(valref_time_stamp_file)

In [53]:
odometry = pri.load_dataset(
    recording, odometry="recording", cache=True
)
odometry.position.hvplot.line(x="time")

:DynamicMap   [cartesian_axis]
   :Curve   [time]   (position)

### Get session start and end from YAML

In [54]:
fps = 30
with open(r'/home/bszekely/Desktop/ProjectsResearch/biomechanics_head_vedb/slippage_sessions_list.yaml') as file:
    documents = yaml.full_load(file)
    for item, doc in documents.items():
        slow_start = doc[session_num]['slow_start']
        slow_end = doc[session_num]['slow_end']
        med_start = doc[session_num]['medium_start']
        med_end = doc[session_num]['medium_end']
        fast_start = doc[session_num]['fast_start']
        fast_end = doc[session_num]['fast_end']

start_i_slow = (slow_start[0][0] * 60 + slow_start[0][1]) * fps
end_i_slow = (slow_end[0][0] * 60 + slow_end[0][1]) * fps

start_i_med = (med_start[0][0] * 60 + med_start[0][1]) * fps
end_i_med = (med_end[0][0] * 60 + med_end[0][1]) * fps

start_i_fast = (fast_start[0][0] * 60 + fast_start[0][1]) * fps 
end_i_fast = (fast_end[0][0] * 60 + fast_end[0][1]) * fps

In [55]:
new_odo = np.zeros((len(world_time_stamp),3))
new_time = np.zeros(len(world_time_stamp))
new_accel= np.zeros((len(world_time_stamp),3))
for i in range(len(world_time_stamp)):   
    odo_index = np.argmin(np.abs((odometry_time_stamp - world_time_stamp[i]).astype(float)))  
    new_odo[i] = odometry.position[odo_index,:]
    new_accel[i] = odometry.linear_acceleration[odo_index]

In [56]:
slow = new_odo[start_i_slow:end_i_slow,:]
med = new_odo[start_i_med:end_i_med,:]
fast = new_odo[start_i_fast:end_i_fast,:]

slow_accel = new_accel[start_i_slow:end_i_slow,:]
med_accel = new_accel[start_i_med:end_i_med,:]
fast_accel = new_accel[start_i_fast:end_i_fast,:]

# peaks_slow = find_peaks_cwt(slow[:,1],np.arange(1,fps))
# peaks_med = find_peaks_cwt(med[:,1],np.arange(1,fps))
# peaks_fast = find_peaks_cwt(fast[:,1],np.arange(1,fps))
peaks_slow, _= find_peaks(slow[:,1],distance=fps / 3)
peaks_med, _= find_peaks(med[:,1],distance=fps / 3)
peaks_fast, _= find_peaks(fast[:,1],distance=fps / 3)

In [57]:
indexes = np.arange(len(fast[:,1]))
# plt.plot(indexes,fast[:,1], '-')
# plt.plot(indexes[peaks_fast], fast[peaks_fast,1],'^')
# plt.show()

# indexes_total = np.arange(len(new_odo))
# plt.plot(indexes_total[start_i_med:end_i_med],new_odo[start_i_med:end_i_med,1],'^')
# plt.plot(indexes_total,new_odo[:,1], '-')
# plt.show()

### Gait variability

this article is used as a reference for the gait variability metric: https://jneuroengrehab.biomedcentral.com/track/pdf/10.1186/1743-0003-2-19.pdf

In [58]:
step_time_slo = np.zeros(len(peaks_slow) - 1)
for i in range(len(peaks_slow) - 1):
     step_time_slo[i] = (peaks_slow[i+1] - peaks_slow[i]) * (1/fps)
        
step_time_med = np.zeros(len(peaks_med) - 1)
for i in range(len(peaks_med) - 1):
     step_time_med[i] = (peaks_med[i+1] - peaks_med[i]) * (1/fps)
        
step_time_fas = np.zeros(len(peaks_fast) - 1)
for i in range(len(peaks_fast) - 1):
     step_time_fas[i] = (peaks_fast[i+1] - peaks_fast[i]) * (1/fps)

# CV of the time  variability
cv_slow = variation(np.abs(np.diff(step_time_slo)))
cv_med = variation(np.abs(np.diff(step_time_med)))
cv_fast = variation(np.abs(np.diff(step_time_fas)))

print(variation(np.abs(np.diff(step_time_slo))))
print(variation(np.abs(np.diff(step_time_med))))
print(variation(np.abs(np.diff(step_time_fas))))

1.0041465278073114
2.3618856443183205
1.289311970284226


### FFT

In [59]:
from scipy.fft import rfft, rfftfreq
N = len(slow[:,1])
yf = rfft(slow[:,1])
xf = rfftfreq(N, 1 / fps)
new_yf = np.delete(yf, 0)
new_xf = np.delete(xf, 0)
real_power = np.abs(new_yf)
max_power = np.max(real_power)
ind = np.where(real_power == max_power)
res_freq = new_xf[ind]
# plt.plot(new_xf, np.abs(new_yf))
# plt.show()

### convert from position to rotation

In [60]:
#head_rb_azimuth_slow = np.rad2deg(np.arctan2(slow[:, 0], slow[:, 2]))
head_rb_elevation_slow = np.rad2deg(np.arctan2(slow[:, 1], slow[:, 2]))
#head_rb_azimuth_med = np.rad2deg(np.arctan2(med[:, 0], med[:, 2]))
head_rb_elevation_med = np.rad2deg(np.arctan2(med[:, 1], med[:, 2]))
#head_rb_azimuth_fast = np.rad2deg(np.arctan2(fast[:, 0], fast[:, 2]))
head_rb_elevation_fast = np.rad2deg(np.arctan2(fast[:, 1], fast[:, 2]))

### Drift

In [61]:
X_slow = np.arange(0,len(slow))
X_slow_ = X_slow.reshape(-1, 1)
X_med = np.arange(0,len(med))
X_med_ = X_med.reshape(-1, 1)
X_fast = np.arange(0,len(fast))
X_fast_ = X_fast.reshape(-1, 1)

reg_slow = LinearRegression().fit(X_slow_, head_rb_elevation_slow)
reg_med = LinearRegression().fit(X_med_, head_rb_elevation_med)
reg_fast = LinearRegression().fit(X_fast_, head_rb_elevation_fast)

reg_slow_accel = LinearRegression().fit(X_slow_, slow_accel[:,1])
reg_med_accel = LinearRegression().fit(X_med_, med_accel[:,1])
reg_fast_accel = LinearRegression().fit(X_fast_, fast_accel[:,1])

# print("slow pitch pos", decimal.Decimal(float(reg_slow.coef_)))
# print("medium pitch pos",decimal.Decimal(float(reg_med.coef_)))
# print("fast pitch pos",decimal.Decimal(float(reg_fast.coef_)))
# print("======================================================")
# print("slow pos accel", decimal.Decimal(float(reg_slow_accel.coef_)))
# print("medium pos accel",decimal.Decimal(float(reg_med_accel.coef_)))
# print("fast pos accel",decimal.Decimal(float(reg_fast_accel.coef_)))

### Jitter

In [62]:
median_window = 60
std_window = 45
scalar = 1
min_window = 10

slow_df_elevation = pd.Series(data=head_rb_elevation_slow)
med_df_elevation = pd.Series(data=head_rb_elevation_med)
fast_df_elevation = pd.Series(data=head_rb_elevation_fast)

slow_df_accel = pd.Series(data=slow_accel[:,1])
med_df_accel = pd.Series(data=med_accel[:,1])
fast_df_accel = pd.Series(data=fast_accel[:,1])

mean_slow_elev = slow_df_elevation.rolling(median_window, win_type='gaussian', min_periods=min_window).mean(std=std_window)
mean_slow_elev = mean_slow_elev.fillna(mean_slow_elev.mean())

mean_med_elev = med_df_elevation.rolling(median_window, win_type='gaussian', min_periods=min_window).mean(std=std_window)
mean_med_elev = mean_med_elev.fillna(mean_med_elev.mean())

mean_fast_elev = fast_df_elevation.rolling(median_window, win_type='gaussian', min_periods=min_window).mean(std=std_window)
mean_fast_elev = mean_fast_elev.fillna(mean_fast_elev.mean())

# Linear acceleration #
mean_slow_accel = slow_df_accel.rolling(median_window, win_type='gaussian', min_periods=min_window).mean(std=std_window)
mean_slow_accel = mean_slow_accel.fillna(mean_slow_accel.mean())

mean_med_accel = med_df_accel.rolling(median_window, win_type='gaussian', min_periods=min_window).mean(std=std_window)
mean_med_accel = mean_med_accel.fillna(mean_med_accel.mean())

mean_fast_accel = fast_df_accel.rolling(median_window, win_type='gaussian', min_periods=min_window).mean(std=std_window)
mean_fast_accel = mean_fast_accel.fillna(mean_fast_accel.mean())

#STD
# print("Jitter elevation - slow:", np.round(mean_slow_elev.std(),3))
# print("Jitter elevation - med:", np.round(mean_med_elev.std(),3))
# print("Jitter elevation - fast:", np.round(mean_fast_elev.std(),3))
# print("======================================================")
# print("Jitter acceleration - slow:", np.round(mean_slow_accel.std(),3))
# print("Jitter acceleration - med:", np.round(mean_med_accel.std(),3))
# print("Jitter acceleration - fast:", np.round(mean_fast_accel.std(),3))

In [63]:
print("Part ID: ",session_num)
print("slow pitch pos drift", decimal.Decimal(float(reg_slow.coef_)))
print("medium pitch pos drift",decimal.Decimal(float(reg_med.coef_)))
print("fast pitch pos drift",decimal.Decimal(float(reg_fast.coef_)))
print("======================================================")
print("slow pos accel drift", decimal.Decimal(float(reg_slow_accel.coef_)))
print("medium pos accel drift",decimal.Decimal(float(reg_med_accel.coef_)))
print("fast pos accel drift",decimal.Decimal(float(reg_fast_accel.coef_)))
print("======================================================")
print("Jitter elevation - slow:", np.round(mean_slow_elev.std(),3))
print("Jitter elevation - med:", np.round(mean_med_elev.std(),3))
print("Jitter elevation - fast:", np.round(mean_fast_elev.std(),3))
print("======================================================")
print("Jitter acceleration - slow:", np.round(mean_slow_accel.std(),3))
print("Jitter acceleration - med:", np.round(mean_med_accel.std(),3))
print("Jitter acceleration - fast:", np.round(mean_fast_accel.std(),3))

Part ID:  2021_07_14_17_30_31
slow pitch pos drift 0.00035406831601038466113273717184029010240919888019561767578125
medium pitch pos drift 0.002187984469595806273634064353927897172980010509490966796875
fast pitch pos drift -0.003828525916258607338094055450028463383205235004425048828125
slow pos accel drift -0.00000992448443874573214655565589481511779013089835643768310546875
medium pos accel drift 0.00002234864026888979345367679252287729241288616321980953216552734375
fast pos accel drift -0.0000227704665058054418042519928544464846709161065518856048583984375
Jitter elevation - slow: 2.086
Jitter elevation - med: 3.112
Jitter elevation - fast: 2.868
Jitter acceleration - slow: 0.071
Jitter acceleration - med: 0.095
Jitter acceleration - fast: 0.179


In [64]:
cols = ["Drift_Slow_Pitch", "Drift_Med_Pitch", "Drift_Fast_Pitch", "Drift_Slow_Accel", "Drift_Med_Accel",
"Drift_Fast_Accel", "Jitter_Slow_Pitch", "Jitter_Med_Pitch", "Jitter_Fast_Pitch", "Jitter_Slow_Accel", 
"Jitter_Med_Accel", "Jitter_Fast_Accel"]

Drift_Slow_Pitch = pd.Series(decimal.Decimal(float(reg_slow.coef_)))
Drift_Med_Pitch = pd.Series(decimal.Decimal(float(reg_med.coef_)))
Drift_Fast_Pitch = pd.Series(decimal.Decimal(float(reg_fast.coef_)))

Drift_Slow_Accel = pd.Series(decimal.Decimal(float(reg_slow_accel.coef_)))
Drift_Med_Accel = pd.Series(decimal.Decimal(float(reg_med_accel.coef_)))
Drift_Fast_Accel = pd.Series(decimal.Decimal(float(reg_fast_accel.coef_)))

Jitter_Slow_Pitch = pd.Series(np.round(mean_slow_elev.std(),3))
Jitter_Med_Pitch = pd.Series(np.round(mean_med_elev.std(),3))
Jitter_Fast_Pitch = pd.Series(np.round(mean_fast_elev.std(),3))

Jitter_Slow_Accel = pd.Series(np.round(mean_slow_accel.std(),3))
Jitter_Med_Accel = pd.Series(np.round(mean_med_accel.std(),3))
Jitter_Fast_Accel = pd.Series(np.round(mean_fast_accel.std(),3))

lst = list([Drift_Slow_Pitch, Drift_Med_Pitch, Drift_Fast_Pitch, Drift_Slow_Accel, Drift_Med_Accel,
          Drift_Fast_Accel, Jitter_Slow_Pitch, Jitter_Med_Pitch, Jitter_Fast_Pitch, Jitter_Slow_Accel,
           Jitter_Med_Accel, Jitter_Fast_Accel])

final_series = pd.concat(lst,axis = 1)
final_series.columns = cols
pwd = os.getcwd()
final_series.to_csv('temp.csv')

In [65]:
# samp_freq =200
# hilbert_signal = hilbert(odometry.position[:,1])
# amplitudes_envelope = np.abs(hilbert_signal)
# instantaneous_phase = np.unwrap(np.angle(hilbert_signal))
# instantaneous_frequency = np.abs((np.diff(instantaneous_phase)/(2.0*np.pi)) * samp_freq) #I am abs the data which messes up the envelope to signal graph.
# fig = plt.figure()
# ax0 = fig.add_subplot(211)
# ax0.plot(odometry.position[:,1], label='signal')
# ax0.plot(amplitudes_envelope, label='envelope')
# ax0.set_xlabel("Samples")
# ax0.legend()
# ax1 = fig.add_subplot(212)
# ax1.plot(instantaneous_frequency)
# ax1.set_xlabel("time in seconds")
# ax1.set_ylim(0, 10)

In [66]:
# iterate = 60 * fps - 1
# for i in range(len(instantaneous_frequency)-iterate):
#     start_p = instantaneous_frequency[[i]]
#     end_p = instantaneous_frequency[[i+iterate]]
#     if start_p >= 0.5 and start_p <= 1.5 and end_p >= 0.5 and end_p <= 1.5:
#         print(i,'start point')
#         print(i+iterate,'end point')